This function computes the (linear) interpolant for evaluating the differential equation:

\begin{equation}
\phi' = - \dfrac{\cos^2(\phi)[C_x^{11}(\mathbf{x})\cos(\phi)+C_y^{11}(\mathbf{x})\sin(\phi)]+[C_x^{12}(\mathbf{x})\cos(\phi)+C_y^{11}(\mathbf{x})\sin(\phi)]\sin(2\phi)+\sin^2(\phi)[C_x^{22}(\mathbf{x})\cos(\phi)+C_y^{22}(\mathbf{x})\sin(\phi)}{\sin(2\phi)[C^{22}(\mathbf{x})-C^{11}(\mathbf{x})]+2\cos(2\phi)C^{12}(\mathbf{x})}
\label{eq: phi_prime}
\end{equation}

at every point $ (\mathbf{x}, \phi) $. 

To do so, we first need to evaluate the right-hand-side of eq. \ref{eq: phi_prime} over a rectangular meshgrid $ X, Y, \phi $ (see *__phi_prime*) before computing the interpolant of $ \dot{\phi} $ (see *_phi_prime*).

| Name | Type (Shape) | Description |
| --- | --- | --- |
| closed_null_geodesics | list (N,) | list containing $ N $ closed null geodesics
| x_elliptic | list | list containing x-coordinates of elliptic OECS
| y_elliptic | list | list containing y-coordinates of elliptic OECS

In [1]:
# Import numpy
import numpy as np

# Import math symbols
from math import cos, sin, pi

# Import numba (used for high performance scientific computing)
from numba import jit, prange

In [1]:
@jit(parallel=True)
def __phi_prime(X, Y, phi, C11, C12, C22, C11_x, C11_y, C12_x, C12_y, C22_x, C22_y):
    
    phi_prime = np.zeros((Y.shape[0], X.shape[1], phi.shape[0]))
    phi_prime_denom = phi_prime.copy()
    
    for i in prange(Y.shape[0]):
        
        for j in prange(X.shape[1]):
            
            C11_ = C11[i, j]
            C12_ = C12[i, j]
            C22_ = C22[i, j]
            
            C11_x_ = C11_x[i, j]
            C11_y_ = C11_y[i, j]
            
            C12_x_ = C12_x[i, j]
            C12_y_ = C12_y[i, j]
            
            C22_x_ = C22_x[i, j]
            C22_y_ = C22_y[i, j]

            for p in prange(phi.shape[0]):
                
                phi_ = phi[p]
    
                phi_prime_num = (cos(phi_)*C11_x_+C11_y_*sin(phi_))*cos(phi_)**2
                phi_prime_num += (cos(phi_)*C12_x_+C12_y_*sin(phi_))*sin(2*phi_)
                phi_prime_num += (cos(phi_)*C22_x_+C22_y_*sin(phi_))*sin(phi_)**2
    
                phi_prime_denom[i, j, p] = sin(2*phi_)*(C22_-C11_)+2*cos(2*phi_)*C12_
            
                phi_prime[i, j, p] = -phi_prime_num/phi_prime_denom[i, j, p]

    return phi_prime, phi_prime_denom

In [2]:
def _phi_prime(X, Y, C11, C12, C22, C11_x, C11_y, C12_x, C12_y, C22_x, C22_y):
    
    phi = np.linspace(0, 2*pi, 360)
    
    # Import interpolator from scipy
    from scipy.interpolate import RegularGridInterpolator
    
    phi_prime, DOE = __phi_prime(X, Y, phi, C11, C12, C22, C11_x, C11_y, C12_x, C12_y, C22_x, C22_y)
    
    return RegularGridInterpolator((Y[:,0], X[0,:], phi), phi_prime), RegularGridInterpolator((Y[:,0], X[0,:], phi), DOE)